<a href="https://colab.research.google.com/github/claudelepere/ML_GitHub/blob/main/Copy_of_Fine_tuning_BERT_(and_friends)_for_multi_label_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tuning BERT (and friends) for multi-label text classification

In this notebook, we are going to fine-tune BERT to predict one or more labels for a given piece of text. Note that this notebook illustrates how to fine-tune a bert-base-uncased model, but you can also fine-tune a RoBERTa, DeBERTa, DistilBERT, CANINE, ... checkpoint in the same way.

All of those work in the same way: they add a linear layer on top of the base model, which is used to produce a tensor of shape (batch_size, num_labels), indicating the unnormalized scores for a number of labels for every example in the batch.



## Set-up environment

First, we install the libraries which we'll use: HuggingFace Transformers and Datasets.

## Load dataset

Next, let's download a multi-label text classification dataset from the [hub](https://huggingface.co/).

At the time of writing, I picked a random one as follows:   

* first, go to the "datasets" tab on huggingface.co
* next, select the "multi-label-classification" tag on the left as well as the the "1k<10k" tag (fo find a relatively small dataset).

Note that you can also easily load your local data (i.e. csv files, txt files, Parquet files, JSON, ...) as explained [here](https://huggingface.co/docs/datasets/loading.html#local-and-remote-files).



file1 = open(r"C:\tmp\BERT_results\zazu.txt", 'w')
L = ["This is Delhi\n", "This is Paris\n", "This is London\n"]
s = "Hello\n"
file1.write(s)
file1.writelines(L)
file1.close()
file1 = open(r"C:\tmp\BERT_results\zazu.txt", 'r')
print(file1.read())
file1.close()



!pwd
!cd /..
!pwd
!ls -la /content/ML_GitHub/BERT_results/checkpoint-80
!cat /content/ML_GitHub/BERT_results/checkpoint-80/trainer_state.json
from google.colab import files
files.download(r"/content/ML_GitHub/BERT_results/checkpoint-80/trainer_state.json")





In [45]:
from getpass import getpass
HF_TOKEN = getpass("Enter your Hugging Face token: ")

Enter your Hugging Face token: ··········


In [46]:
from huggingface_hub import login
login(token=HF_TOKEN, add_to_git_credential=True)

Token is valid (permission: fineGrained).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [47]:
import os, sys, shutil

os.chdir("/content")
current_dir = os.getcwd()
print(f"The current directory is {current_dir}")

if os.path.isdir('ML_GitHub'):
    shutil.rmtree('ML_GitHub')

!git init
!git branch -m main
!git clone https://github.com/claudelepere/ML_GitHub.git
print(f"list /content: {os.listdir(current_dir)}")
print(f"list /content/ML_GitHub: {os.listdir(current_dir + '/ML_GitHub')}")
!ls -la /content/ML_GitHub/datasetHF
os.chdir(current_dir + '/ML_GitHub')

#!pip install fsspec==2024.10.0
!pip install -q transformers datasets
from datasets import DatasetDict

dataset = DatasetDict.load_from_disk('datasetHF')
print(f"dataset: {type(dataset)} {dataset.shape}\n{dataset}")

The current directory is /content
Reinitialized existing Git repository in /content/.git/
Cloning into 'ML_GitHub'...
remote: Enumerating objects: 89, done.
remote: Counting objects: 100% (89/89), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 89 (delta 45), reused 52 (delta 23), pack-reused 0 (from 0)
Receiving objects: 100% (89/89), 10.03 MiB | 14.69 MiB/s, done.
Resolving deltas: 100% (45/45), done.
list /content: ['.config', '.git', 'ML_GitHub', 'sample_data']
list /content/ML_GitHub: ['.git', 'datasetHF', 'README.md', 'Copy_of_Fine_tuning_BERT_(and_friends)_for_multi_label_text_classification.ipynb']
total 24
drwxr-xr-x 5 root root 4096 Nov 11 12:15 .
drwxr-xr-x 4 root root 4096 Nov 11 12:15 ..
-rw-r--r-- 1 root root   43 Nov 11 12:15 dataset_dict.json
drwxr-xr-x 2 root root 4096 Nov 11 12:15 test
drwxr-xr-x 2 root root 4096 Nov 11 12:15 train
drwxr-xr-x 2 root root 4096 Nov 11 12:15 validation
dataset: <class 'datasets.dataset_dict.DatasetDict'> {'train': (6

As we can see, the dataset contains 3 splits: one for training, one for validation and one for testing.

Let's check the first example of the training split:

In [48]:
example = dataset['validation'][0]
#example

The dataset consists of texts, labeled with one or more skills.

Let's create a list that contains the labels, as well as 2 dictionaries that map labels to integers and back.

In [49]:
labels = [label for label in dataset['train'].features.keys() if label not in ['id', 'text']]
labels.sort()

print(len(labels), list(enumerate(labels)))
id2label = {idx:label for idx, label in enumerate(labels)}
print(id2label)
label2id = {label:idx for idx, label in enumerate(labels)}
print(len(labels), labels)

48 [(0, '142'), (1, '143'), (2, '146'), (3, '147'), (4, '148'), (5, '149'), (6, '150'), (7, '151'), (8, '152'), (9, '153'), (10, '154'), (11, '155'), (12, '156'), (13, '157'), (14, '158'), (15, '160'), (16, '162'), (17, '165'), (18, '167'), (19, '168'), (20, '169'), (21, '170'), (22, '171'), (23, '173'), (24, '174'), (25, '175'), (26, '176'), (27, '356'), (28, '360'), (29, '361'), (30, '362'), (31, '364'), (32, '371'), (33, '373'), (34, '375'), (35, '376'), (36, '394'), (37, '408'), (38, '409'), (39, '667'), (40, '685'), (41, '686'), (42, '689'), (43, '756'), (44, '757'), (45, '758'), (46, '760'), (47, '761')]
{0: '142', 1: '143', 2: '146', 3: '147', 4: '148', 5: '149', 6: '150', 7: '151', 8: '152', 9: '153', 10: '154', 11: '155', 12: '156', 13: '157', 14: '158', 15: '160', 16: '162', 17: '165', 18: '167', 19: '168', 20: '169', 21: '170', 22: '171', 23: '173', 24: '174', 25: '175', 26: '176', 27: '356', 28: '360', 29: '361', 30: '362', 31: '364', 32: '371', 33: '373', 34: '375', 35: '3

## Preprocess data

As models like BERT don't expect text as direct input, but rather `input_ids`, etc., we tokenize the text using the tokenizer. Here I'm using the `AutoTokenizer` API, which will automatically load the appropriate tokenizer based on the checkpoint on the hub.

What's a bit tricky is that we also need to provide labels to the model. For multi-label text classification, this is a matrix of shape (batch_size, num_labels). Also important: this should be a tensor of floats rather than integers, otherwise PyTorch' `BCEWithLogitsLoss` (which the model will use) will complain, as explained [here](https://discuss.pytorch.org/t/multi-label-binary-classification-result-type-float-cant-be-cast-to-the-desired-output-type-long/117915/3).

In [50]:
from transformers import AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# examples and not example because batched=True => examples is a batch
def preprocess_data(examples, indices):
  # take a batch of texts
  text = examples["text"]
  #print("Indices:", indices, "Batch size:", len(text), "Num labels:", len(labels))

  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=512)

  labels_batch = {label: np.zeros(len(text)) for label in labels}                                                # validation: 42 rows: 'label': 18 x 0.0
  #print(f"labels_batch: zeros: {type(labels_batch)} {len(labels_batch)} {labels_batch}")
  #print(f"examples.keys(): {examples.keys()}")
  # examples.keys(): all the column names (keys) in the batch dict "examples"
  # for each k in labels, a new key-value pair k: examples[k] is added to labels_batch
  labels_batch.update({k: [1.0 if val else 0.0 for val in examples[k]] for k in examples.keys() if k in labels}) # validation: 42 rows: 'label': 1.0 or 0.0

  #print(f"labels_batch: updated: {type(labels_batch)} {len(labels_batch)} {labels_batch}")

  # create numpy array of shape (batch_size, num_labels)
  labels_matrix = np.zeros((len(text), len(labels)))                                                             # validation: 18 rows 42 columns
  #print(f"labels_matrix:{type(labels_matrix)} {labels_matrix.shape}")
  # fill numpy array
  for idx, label in enumerate(labels):
    #print(f"idx:{idx} label:{label}")                                                                            # labels are sorted
    labels_matrix[:, idx] = labels_batch[label] # for each row, sets the idx-th column of labels_matrix with the values from labels_batch[label]

  #print("First row of labels_matrix:", labels_matrix[0])

  # Add labels to encoding
  encoding["labels"] = labels_matrix.tolist()
  print(f"encoding['labels']: {encoding['labels']}")


  return encoding



/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [51]:
encoded_dataset = dataset.map(preprocess_data, batched=True, remove_columns=dataset['train'].column_names, with_indices=True)

Map:   0%|          | 0/640 [00:00<?, ? examples/s]

encoding['labels']: [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 0.0, 

Map:   0%|          | 0/90 [00:00<?, ? examples/s]

encoding['labels']: [[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 

Map:   0%|          | 0/270 [00:00<?, ? examples/s]

encoding['labels']: [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 

In [52]:
example = encoded_dataset['validation'][0]
print(f"example.keys(): {example.keys()}")
print(f"example['input_ids']: {example['input_ids']}")
print(f"example['token_type_ids']: {example['token_type_ids']}")
print(f"example['attention_mask']: {example['attention_mask']}")
print(f"example['labels']: {example['labels']}")

example.keys(): dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
example['input_ids']: [101, 24869, 2361, 11968, 18410, 2015, 3481, 2483, 1011, 4646, 3036, 6739, 9262, 1010, 2394, 1010, 3036, 24869, 2361, 11968, 18410, 2015, 3481, 2483, 2215, 2000, 2393, 4338, 1996, 2924, 1997, 4826, 2651, 1029, 2106, 2017, 2412, 4687, 2129, 2057, 5676, 2008, 5097, 1999, 1996, 2924, 2024, 5851, 1998, 3961, 5851, 1029, 2306, 1996, 24873, 3036, 1010, 1996, 4646, 3036, 1004, 18130, 2968, 2136, 21312, 2006, 1037, 3679, 3978, 2008, 2035, 5097, 1997, 1996, 2924, 2468, 1998, 3961, 5851, 1012, 1999, 2344, 2000, 2079, 2023, 1010, 2057, 16437, 2731, 19817, 13006, 22471, 18909, 2005, 9797, 2006, 7882, 3036, 7832, 1998, 2490, 1996, 9797, 1999, 13729, 1996, 3036, 3314, 2027, 8087, 2007, 2037, 2764, 3642, 1012, 2320, 2838, 2024, 3201, 2005, 3048, 2875, 2537, 1010, 2057, 10939, 1998, 15389, 20015, 5604, 2006, 1996, 2838, 2000, 5676, 2008, 1996, 2924, 3640, 5851, 8169, 8360, 6447, 2875, 2049, 801

In [53]:
tokenizer.decode(example['input_ids'])

"[CLS] bnp paribas fortis - application security expert java, english, security bnp paribas fortis want to help shape the bank of tomorrow today? did you ever wonder how we ensure that applications in the bank are secure and remain secure? within the coe security, the application security & vulnerability management team ensures on a daily basis that all applications of the bank become and remain secure. in order to do this, we setup training trajectories for developers on relevant security topics and support the developers in solving the security issues they encounter with their developed code. once features are ready for moving towards production, we organize and execute penetration testing on the features to ensure that the bank provides secure banking functionalities towards its customer. finally, we constantly search for vulnerabilities, through extensive scanning, on the applications and the infrastructure of the bank, once identified we inform the involved it teams so that they c

In [54]:
example = encoded_dataset['validation'][0]
print(f"example['labels']: {example['labels']}")


example['labels']: [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [55]:
[id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]


['146']

Finally, we set the format of our data to PyTorch tensors. This will turn the training, validation and test sets into standard PyTorch [datasets](https://pytorch.org/docs/stable/data.html).

In [56]:
encoded_dataset.set_format("torch")
example = encoded_dataset['validation'][0]
print(f"example['labels']:  {type(example['labels'])} {example['labels'].shape}\n{example['labels']}")

#raise Exception("STOP")

example['labels']:  <class 'torch.Tensor'> torch.Size([48])
tensor([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])


## Define model

Here we define a model that includes a pre-trained base (i.e. the weights from bert-base-uncased) are loaded, with a random initialized classification head (linear layer) on top. One should fine-tune this head, together with the pre-trained base on a labeled dataset.

This is also printed by the warning.

We set the `problem_type` to be "multi_label_classification", as this will make sure the appropriate loss function is used (namely [`BCEWithLogitsLoss`](https://pytorch.org/docs/stable/generated/torch.nn.BCEWithLogitsLoss.html)). We also make sure the output layer has `len(labels)` output neurons, and we set the id2label and label2id mappings.

In [57]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased",
                                                           problem_type="multi_label_classification",
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [58]:
#raise Exception("STOP")

## Train the model!

We are going to train the model using HuggingFace's Trainer API. This requires us to define 2 things:

* `TrainingArguments`, which specify training hyperparameters. All options can be found in the [docs](https://huggingface.co/transformers/main_classes/trainer.html#trainingarguments). Below, we for example specify that we want to evaluate after every epoch of training, we would like to save the model every epoch, we set the learning rate, the batch size to use for training/evaluation, how many epochs to train for, and so on.
* a `Trainer` object (docs can be found [here](https://huggingface.co/transformers/main_classes/trainer.html#id1)).

In [59]:
batch_size = 8
metric_name = "f1"

In [60]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    output_dir = r'C:\tmp\BERT_results\output',
    overwrite_output_dir=True,
    logging_dir= r'C:\tmp\BERT_results\logs',
    logging_steps=50,
    save_steps=100,
    save_total_limit=2,
    eval_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    #push_to_hub=True,
)

We are also going to compute metrics while training. For this, we need to define a `compute_metrics` function, that returns a dictionary with the desired metric values.

In [61]:
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score, accuracy_score, average_precision_score
from transformers import EvalPrediction
import torch

# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.2):
    _average = 'weighted'    # 'micro' or 'weighted'
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true               = labels

    f1                   = f1_score               (y_true=y_true, y_pred=y_pred, average=_average, zero_division=1)
    precision            = precision_score        (y_true=y_true, y_pred=y_pred, average=_average, zero_division=1)
    recall               = recall_score           (y_true=y_true, y_pred=y_pred, average=_average, zero_division=1)
    #roc_auc              = roc_auc_score          (y_true=y_true, y_score=probs, average=_average)
    precision_recall_auc = average_precision_score(y_true=y_true, y_score=probs, average=_average)
    accuracy             = accuracy_score         (y_true=y_true, y_pred=y_pred)
    # return as dictionary
    metrics = {'f1'                  : f1,
               'precision'           : precision,
               'recall'              : recall,
               #'roc_auc'             : roc_auc,
               'precision_recall_auc': precision_recall_auc,
               'accuracy'            : accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds,
        labels=p.label_ids)
    return result

Let's verify a batch as well as a forward pass:

In [62]:
encoded_dataset['train'][0]['labels'].type()

'torch.FloatTensor'

In [63]:
encoded_dataset['train']['input_ids'][0]

tensor([  101, 29454, 22684,  1011, 11917, 23604, 27333,  2368,  1010, 23604,
        29454, 22684, 10709,  4436,  2000,  5454,  2005, 29454, 22684,  2079,
         2017,  2215,  2000, 12992,  2115,  2476,  1029,  2024,  2017,  9461,
         2000,  4553,  1998,  2079,  2017,  2215,  2000, 18759,  3112,  1029,
         2065,  2061,  1010,  2017,  1005,  2128,  2012,  1996,  2157,  4769,
         1012,  2012, 29454, 22684,  1010,  2362,  1010,  2057,  2328,  2115,
         2925,   999,  2005,  2256,  2047,  2622,  2057,  1005,  2128,  2559,
         2005,  2017,  1012,  2017,  1005,  2128,  2619,  2040,  2038,  2019,
         2398,  1011,  2006,  5177,  3012,  1010,  2017,  2066,  2108,  2006,
         1996,  2346,  1010,  2021,  2926,  2017,  8116,  2152,  3737,  2147,
         1012,  1037,  3020,  3014,  2003,  2025,  6827,  1012,  2057,  3749,
         2017,  1037,  2731,  2000,  2468,  1037,  7378, 11917,  2057, 16502,
          999,  3105,  3141,  4813,  2017,  1005,  2128, 19376, 

In [64]:
#forward pass
print(f"inputids: {type(encoded_dataset['train']['input_ids'][0])} {encoded_dataset['train']['input_ids'][0].shape}")
print(f"attention_mask: {type(encoded_dataset['train']['attention_mask'][0])} {encoded_dataset['train']['attention_mask'][0].shape}")
print(f"labels: {type(encoded_dataset['train'][0]['labels'])} {encoded_dataset['train'][0]['labels'].shape}")

outputs = model(input_ids=encoded_dataset['train']['input_ids'][0].unsqueeze(0),
                attention_mask=encoded_dataset['train']['attention_mask'][0].unsqueeze(0),
                labels=encoded_dataset['train'][0]['labels'].unsqueeze(0))
outputs

inputids: <class 'torch.Tensor'> torch.Size([512])
attention_mask: <class 'torch.Tensor'> torch.Size([512])
labels: <class 'torch.Tensor'> torch.Size([48])


SequenceClassifierOutput(loss=tensor(0.7021, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), logits=tensor([[ 0.3957,  0.6405,  0.0656, -0.4180,  0.3979,  0.6778, -0.5049, -0.2096,
          0.0352,  0.0264,  0.4794, -0.0714, -0.1936,  0.1782,  0.0213, -0.1550,
         -0.2312,  0.3284,  0.5978, -0.0082, -0.3412, -0.1144, -0.2776, -0.0838,
         -0.7082, -0.0832,  0.1135, -0.5408, -0.3100, -0.3048, -0.1134,  0.0868,
         -0.0692, -0.0583, -0.4026,  0.0869, -0.4054,  0.0949,  0.4201, -0.5702,
         -0.0797,  0.2068, -0.1037,  0.2270, -0.3648,  0.5192,  0.1924,  0.0604]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

Let's start training!

In [65]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [66]:
trainer.train()

Epoch,Training Loss,Validation Loss,F1,Precision,Recall,Precision Recall Auc,Accuracy
1,0.566600,0.398579,0.183465,0.117381,0.977654,0.148183,0.000000
2,0.343200,0.302176,0.170437,0.278624,0.793296,0.172280,0.000000
3,0.296100,0.258612,0.141652,0.519453,0.547486,0.217239,0.000000
4,0.252400,0.239080,0.110564,0.691930,0.374302,0.226245,0.000000
5,0.235800,0.233418,0.110564,0.691930,0.374302,0.234996,0.000000


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_ranking.py:1030: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_ranking.py:1030: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_ranking.py:1030: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_ranking.py:1030: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_ranking.py:1030: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_ranking.py:1030: UserWarning: No positive c

TrainOutput(global_step=400, training_loss=0.3275557518005371, metrics={'train_runtime': 395.2709, 'train_samples_per_second': 8.096, 'train_steps_per_second': 1.012, 'total_flos': 842303117721600.0, 'train_loss': 0.3275557518005371, 'epoch': 5.0})

In [67]:
trainer.save_model("my_model")    # Save the trained model and tokenizer: saves the model weights, the tokenizer, the model configuration file ("config.json")

import json

with open("training_metrics.json", 'w') as f:
    json.dump(trainer.state.log_history,f)


## Evaluate

After training, we evaluate our model on the validation set.

In [68]:
eval_results = trainer.evaluate()

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_ranking.py:1030: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_ranking.py:1030: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_ranking.py:1030: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_ranking.py:1030: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_ranking.py:1030: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_ranking.py:1030: UserWarning: No positive c

In [69]:
with open("eval_metrics.json", 'w') as f:
    json.dump(eval_results, f)

In [71]:
from google.colab import files

#!zip -r /content/ML_GitHub/BERT_results.zip /content/ML_GitHub/BERT_results
#files.download("/content/ML_GitHub/BERT_results.zip")
!zip -r model.zip my_model
files.download("model.zip")
files.download("training_metrics.json")
files.download("eval_metrics.json")

raise Exception("STOP")

updating: my_model/ (stored 0%)
updating: my_model/config.json (deflated 63%)
updating: my_model/tokenizer.json (deflated 71%)
updating: my_model/training_args.bin (deflated 51%)
updating: my_model/model.safetensorshf_IJxEnLKMoKzYCytwrRejZOJIdbysSkdANL
 (deflated 7%)
updating: my_model/special_tokens_map.json (deflated 42%)
updating: my_model/tokenizer_config.json (deflated 76%)
updating: my_model/vocab.txt (deflated 53%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Exception: STOP

## Inference

Let's test the model on a new sentence:

id: 323697
"Voor een klant van Talencia ben ik opzoek naar een Senior Full Stack Developer (Java & Angular) Job beschrijving Als Developer zal je een bestaand team toevoegen en meewerken aan de buitbouw van webapplicaties op Azure. Dit is om bestaande applicaties te vervangen die end-of-live zijn. Het project is al in volle realisatie. Profiel Zeer goede kennis van Java en Angular Goede kennis van Azure DevOps, AKS,.. is een grote pluspunt Kennis van Docker/ SQL/ OAuth/PWA/ RESTful API is vereist Taal: Nederlands met kennis van Engels Extra informatie Teamspeler met ervaring in Agile methodiek is vereist. Als je meer informatie wilt en dit klinkt interessant voor u, aarzel dan niet om uw meest recente CV door te sturen. Het kan zijn dat ik niet beschik over uw meest recente CV en dat ik daarom u deze opportuniteit doorstuur dat niet geschikt is voor u. Als u iemand kent dat deze missie interessant zou vinden mag u deze vacature doorsturen. Met vriendelijke groeten,"

['142', '147', '149', '154', '156', '157', '173', '409', '685', '689']

---

id: 323611,"Atcon Global - Project Management Officer / PMO team management Atcon Global For one of our clients, we are looking for an experienced Project Management Officer (PMO) / Project Manager (PM) for permanent employment in the Flanders region. Your role? As a PMO, you will play a crucial role in setting up and improving our project management processes. You will not only be responsible for developing PM standards, but also for carrying out projects independently as a Project Manager. Your duties and responsibilities will include: Developing PMO and project management standards Executing and managing complex digital projects Oversee project progress and report to senior management Follow-up of project budgets, project selection, capacity planning and resource management Coaching and training project managers Identifying and managing project risks Promote continuous improvement in the project management domain Collaborate with stakeholders and external partners Who are we looking for? Bachelor's or master's degree 5+ years in a similar role in a dynamic organization Expertise in project management methods (Agile, Scrum, Lean, Kanban) Strong analytical and problem-solving skills Excellent communication and stakeholder management Experience in team management with clear objectives Proactive, Hands-on mentality and result-oriented Fluent in Dutch and English; French is a plus What's on offer? A dynamic and varied role in a growing, ambitious and innovative company Numerous opportunities for personal growth and career development A competitive salary with customizable benefits A friendly, collegial working atmosphere Flexible working hours, possibility to work from home","171,170,794,800,798,797,138,139,352"

---


In [ ]:
#text = "Voor een klant van Talencia ben ik opzoek naar een Senior Full Stack Developer (Java & Angular) Job beschrijving Als Developer zal je een bestaand team toevoegen en meewerken aan de buitbouw van webapplicaties op Azure. Dit is om bestaande applicaties te vervangen die end-of-live zijn. Het project is al in volle realisatie. Profiel Zeer goede kennis van Java en Angular Goede kennis van Azure DevOps, AKS,.. is een grote pluspunt Kennis van Docker/ SQL/ OAuth/PWA/ RESTful API is vereist Taal: Nederlands met kennis van Engels Extra informatie Teamspeler met ervaring in Agile methodiek is vereist. Als je meer informatie wilt en dit klinkt interessant voor u, aarzel dan niet om uw meest recente CV door te sturen. Het kan zijn dat ik niet beschik over uw meest recente CV en dat ik daarom u deze opportuniteit doorstuur dat niet geschikt is voor u. Als u iemand kent dat deze missie interessant zou vinden mag u deze vacature doorsturen. Met vriendelijke groeten"
text = "Atcon Global - Project Management Officer / PMO team management Atcon Global For one of our clients, we are looking for an experienced Project Management Officer (PMO) / Project Manager (PM) for permanent employment in the Flanders region. Your role? As a PMO, you will play a crucial role in setting up and improving our project management processes. You will not only be responsible for developing PM standards, but also for carrying out projects independently as a Project Manager. Your duties and responsibilities will include: Developing PMO and project management standards Executing and managing complex digital projects Oversee project progress and report to senior management Follow-up of project budgets, project selection, capacity planning and resource management Coaching and training project managers Identifying and managing project risks Promote continuous improvement in the project management domain Collaborate with stakeholders and external partners Who are we looking for? Bachelor's or master's degree 5+ years in a similar role in a dynamic organization Expertise in project management methods (Agile, Scrum, Lean, Kanban) Strong analytical and problem-solving skills Excellent communication and stakeholder management Experience in team management with clear objectives Proactive, Hands-on mentality and result-oriented Fluent in Dutch and English; French is a plus What's on offer? A dynamic and varied role in a growing, ambitious and innovative company Numerous opportunities for personal growth and career development A competitive salary with customizable benefits A friendly, collegial working atmosphere Flexible working hours, possibility to work from home"
encoding = tokenizer(text, return_tensors="pt")
encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

outputs = trainer.model(**encoding)

The logits that come out of the model are of shape (batch_size, num_labels). As we are only forwarding a single sentence through the model, the `batch_size` equals 1. The logits is a tensor that contains the (unnormalized) scores for every individual label.

In [ ]:
logits = outputs.logits
logits.shape

To turn them into actual predicted labels, we first apply a sigmoid function independently to every score, such that every score is turned into a number between 0 and 1, that can be interpreted as a "probability" for how certain the model is that a given class belongs to the input text.

Next, we use a threshold (typically, 0.5) to turn every probability into either a 1 (which means, we predict the label for the given example) or a 0 (which means, we don't predict the label for the given example).

In [ ]:
# apply sigmoid + threshold
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(logits.squeeze().cpu())
predictions = np.zeros(probs.shape)
predictions[np.where(probs >= 0.5)] = 1
# turn predicted id's into actual label names
predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]
print(predicted_labels)

**id**: 323697

**MySQL**: "142,189,190,754,208,794,676,811,812,139,138" (only 142="Developer / Analyst Programmer") is a 7-skill)

**predicted_labels**: ['148', '152', '154', '409'] : all are 7-skills: 148="Technical Analyst", 152="Technical Writer", 154="Database Admininistrator", 409="SOA Specialist"